![Chisel](https://chisel.eecs.berkeley.edu/assets/img/chisel_64.png)

# Module 3.2: Generators: Collections

#### Written by Chick Markley, Stevo Bailey, Edward Wang, and Richard Lin
[chick@berkeley.edu](mailto:chick@berkeley.edu),
[stevo@berkeley.edu](mailto:stevo@berkeley.edu),
[edwardw@berkeley.edu](mailto:edwardw@berkeley.edu), [richard.lin@berkeley.edu](mailto:edwardw@berkeley.edu)

## Table of Contents

1. [Generators and Collections](#generators-and-collections)
1. [Create a FIR Golden Model](#fir-golden-model)
1. [Testing the Golden Model](#test-golden-model)
1. [Now base the test on the Golden Model](#use-golden-model-as-test)
1. [Optional Exercise ](#optional-exercise)


## Set Up

In [67]:
val path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling Main.sc


path: String = "/Volumes/UCB-BAR/generator-bootcamp/load-ivy.sc"

In [68]:
import chisel3._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester, TesterOptionsManager}
import chisel3.util._

import chisel3._

import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester, TesterOptionsManager}

import chisel3.util._

## Generators and Collections<a name="generators-and-collections"></a> 
In this section we will focus on the concept of *generators* and the use of scala collections as a tool to implement them.
Instead of looking at Chisel code as an *instance* of circuit, i.e. a description of a particular circuit, 
we will instead consider it to be a generator of a circuit.

We will start by considering the FIR filter from previous exercises.  

In [69]:
class My4ElementFir(b0: Int, b1: Int, b2: Int, b3: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
  })

  val x_n1 = RegNext(io.in, 0.U)
  val x_n2 = RegNext(x_n1, 0.U)
  val x_n3 = RegNext(x_n2, 0.U)
  io.out := io.in * b0.U(8.W) + x_n1 * b1.U(8.W) +
    x_n2 * b2.U(8.W) + x_n3 * b3.U(8.W)
}


defined class My4ElementFir

This circuit is a simple case of a generator because it can generate versions of this 4-tap filter with
different coefficients.  But what if we want the circuit to have more taps.  We are going to do this in several steps.

- Build a software *Golden Model* of a tap configurable FIR.
- Redesign our test to use this model, and confirm that it works.
- Refactor our My4ElementFir to allow an configurable number of taps.
- Test the new circuit using our new test harness



## An FIR Golden Model<a name="fir-golden-model"></a> 
Below is a Scala software implementation of a FIR circuit.  

In [70]:
/**
  * A naive implementation of an FIR filter with an arbitrary number of taps.
  */
class ScalaFirFilter(taps: Seq[Int]) {
  var pseudoRegisters = List.fill(taps.length)(0)

  def poke(value: Int): Int = {
    pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)
    var accumulator = 0
    for(i <- taps.indices) {
      accumulator += taps(i) * pseudoRegisters(i)
    }
    accumulator
  }
}

defined class ScalaFirFilter

### Seq
Note that **taps** has become a **Seq[Int]** which means that the user of the class can pass a *Seq* or sequence of arbitrary length when constructing the class.
### Registers
With `  var pseudoRegisters = List.fill(taps.length)(0)` we create a `List` that will hold the values from previous cycles.  `List` was chosen because it syntax of adding an element to the head and remove the last element is very simple. Just about any member of the scala collections family could be used.  We are also initializing this list to contain all zeros.
### Poke
Our class adds a poke function/method that emulates putting a new input into the filter and cycling the clock.
### Updating the registers
The line `pseudoRegisters = value :: pseudoRegisters.take(taps.length - 1)` first uses the `take` method of list to keep the all but the last element of the list, then uses the `::` list concatentation operator to add `value` to the head of the reduced version of the list.
### Computing the output
A simple for loop with an accumulator that sums the multiplication of each element of the list times the corresponding tap coefficient. The line with just *accumulator* returns that value as the function result.

## Adapting our previous test for testing our golden model<a name="test-golden-model"></a> 
We will now use our previous work to confirm that our golden model works.  A bit of 
editing magic takes our previous tests harness and morphs it into

In [71]:
val filter = new ScalaFirFilter(Seq(1, 1, 1, 1))

var out = 0

out = filter.poke(1)
println(s"out = $out")
assert(out == 1)  // 1, 0, 0, 0

out = filter.poke(4)
assert(out == 5)  // 4, 1, 0, 0
println(s"out = $out")

out = filter.poke(3)
assert(out == 8)  // 3, 4, 1, 0
println(s"out = $out")

out = filter.poke(2)
assert(out == 10)  // 2, 3, 4, 1
println(s"out = $out")

out = filter.poke(7)
assert(out == 16)  // 7, 2, 3, 4
println(s"out = $out")

out = filter.poke(0)
assert(out == 12)  // 0, 7, 2, 3
println(s"out = $out")

out = 1
out = 5
out = 8
out = 10
out = 16
out = 12


filter: ScalaFirFilter = $sess.cmd69Wrapper$Helper$ScalaFirFilter@7ec16f39
out: Int = 12

Executing the previous block demonstrates that our software model returns the same results as My4ElementFir did.


## Rebuilding our test to use our golden model.<a name="use-golden-model-as-test"></a> 
Now that we are reasonably confident with our golden model we re-write our test that compares 
the circuit outputs with the output of the golden model, instead of using laboriously hand-crafted
examples.
What follows is a quick first pass to do it.

In [72]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new My4ElementFir(1, 1, 1, 1)) {
    c => new PeekPokeTester(c) {
        for(i <- 0 until 100) {
            val input = scala.util.Random.nextInt(8)
          
            val goldenModelResult = goldenModel.poke(input)

            poke(c.io.in, input)
          
            expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

            step(1)
        }
    }
}

[info] [0.001] Elaborating design...
[info] [0.020] Done elaborating.
Total FIRRTL Compile Time: 21.9 ms
Total FIRRTL Compile Time: 5.4 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1512154063460
test cmd68WrapperHelperMy4ElementFir Success: 100 tests passed in 105 cycles taking 0.015427 seconds
[info] [0.013] RAN 100 CYCLES PASSED


goldenModel: ScalaFirFilter = $sess.cmd69Wrapper$Helper$ScalaFirFilter@6cb022ed
res71_1: Boolean = true

Now our test runs 100 cycles, and checks that at each step the two different methods, hardware and software, are in sync.

### Things to watch out for
> Actually things that I screwed up while writing this.

1. Getting the step in the right place. Software and hardware execute differently, it's easy to get this wrong
1. This test is weak because it is very sensitive to how the IOs and registers are sized. Implementing a software golden model that observes wrapping behavior at arbitrary data bit widths can be complicated.  Here I just make sure that I only pass in values that fit.

### Parameterizing the number of taps
Below we have created a new Filter class, `MyManyElementsFilter` that takes a `Seq` of constants to use for taps.  This list can be any number of elements.
For good measure a `bitWidth` has been added that allows us to control the sizes of numbers that can be handled by our circuit.
In response the variable length we have had to refactor the creation of registers and how they are connected.
The methodology used below uses a simple subset of the available library of collection functions.
Later sections show how to more succinctly express the behavior in a way that also makes it clearer what is happening.

In [73]:
class MyManyElementFir(consts: Seq[Int], bitWidth: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(bitWidth.W))
    val out = Output(UInt(bitWidth.W))
  })

  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      muls += regs(i) * consts(i).U
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until consts.length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan.last
}

defined class MyManyElementFir

#### How did we do it.
There are three parallel sections starting at lines 7, 13, and 18.
We are using a Scala collection type called `ArrayBuffer`.
`ArrayBuffer` allows you to append elements using the `+=` operator (also insert and delete, but we don't need this).
First, we create an ArrayBuffer *regs* whose elements will be UInts.
Then iterate over the taps, adding the input as the first element followed by creating registers using RegNext which connects the input of the registere to the previous element `regs(i - 1)` and initalizes the it to unsigned zero `0.U`.
These registers will hold the previous values of inputs as they are needed.

Next, we create another ArrayBuffer *muls* of UInts.
Each element of muls will be a node whose i-th element is the product of the `regs(i)` and `const(i)`

>Note the use of the `scan.last` method.
It takes the last element of a collection, and is a more elegant alternative to `regs(i - 1)` used during the `regs` construction.

### Does it work the same as `My4ElementFir`
A good first test of our new version is to see if it can pass the test we just applied to the
`My4ElementFir`.
We create an instance of `MyManyElementFir` and run even more data through it.

In [74]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new MyManyElementFir(Seq(1, 1, 1, 1), 8)) {
    c => new PeekPokeTester(c) {
        for(i <- 0 until 100) {
            val input = scala.util.Random.nextInt(8)
          
            val goldenModelResult = goldenModel.poke(input)

            poke(c.io.in, input)
          
            expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

            step(1)
        }
    }
}

[info] [0.000] Elaborating design...
[info] [0.006] Done elaborating.
Total FIRRTL Compile Time: 3.0 ms
Total FIRRTL Compile Time: 2.6 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1512154071502
test cmd72WrapperHelperMyManyElementFir Success: 100 tests passed in 105 cycles taking 0.009417 seconds
[info] [0.009] RAN 100 CYCLES PASSED


goldenModel: ScalaFirFilter = $sess.cmd69Wrapper$Helper$ScalaFirFilter@776e9689
res73_1: Boolean = true

### Now let's test a bunch of different sized FIR filters
We create a couple of convenience functions: `r` which gets a random number; `runOneTest` which will create a golden model and a hardware simulation of a filter for a particular set of taps and then run at least twice the number of taps worth of data through the filter.

In [75]:
/** a convenience method to get a random integer
  */
def r(): Int = {
    scala.util.Random.nextInt(1024)
}

/**
  * run a test comparing software and hardware filters
  * run for at least twice as many samples as taps
  */
def runOneTest(taps: Seq[Int]) {
    val goldenModel = new ScalaFirFilter(taps)

    Driver(() => new MyManyElementFir(taps, 32)) {
        c => new PeekPokeTester(c) {
            for(i <- 0 until 2 * taps.length) {
                val input = r()

                val goldenModelResult = goldenModel.poke(input)

                poke(c.io.in, input)

                expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

                step(1)
            }
        }
    }
}

for(tapSize <- 2 until 100 by 10) {
    val taps = Seq.fill(tapSize)(r())  // create a sequence of random coefficients
    
    runOneTest(taps)
}

[info] [0.000] Elaborating design...
[info] [0.001] Done elaborating.
Total FIRRTL Compile Time: 3.4 ms
Total FIRRTL Compile Time: 2.6 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1512154073243
test cmd72WrapperHelperMyManyElementFir Success: 4 tests passed in 9 cycles taking 0.005635 seconds
[info] [0.005] RAN 4 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 8.6 ms
Total FIRRTL Compile Time: 6.3 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1512154073262
test cmd72WrapperHelperMyManyElementFir Success: 24 tests passed in 29 cycles taking 0.016246 seconds
[info] [0.015] RAN 24 CYCLES PASSED
[info] [0.000] Elaborating design...
[info] [0.002] Done elaborating.
Total FIRRTL Compile Time: 13.3 ms
Total FIRRTL Compile Time: 13.1 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1512154073306
test cmd72WrapperHelperMyManyElementFir Success: 44 tests passed in 49 

defined function r
defined function runOneTest

### Just for fun let's make a bigger one
The following will run a single test on a 1000 tap long
FIR filter.  It can take a minute or so to run.
(Hint: Watch for the Scala ● to change to Scala ○ ont the Toolbar when the execution completes)

In [76]:
runOneTest(Seq.fill(1000)(r()))

[info] [0.000] Elaborating design...
[info] [0.016] Done elaborating.
Total FIRRTL Compile Time: 418.7 ms
Total FIRRTL Compile Time: 343.9 ms
End of dependency graph
Circuit state created
[info] [0.000] SEED 1512154077159
test cmd72WrapperHelperMyManyElementFir Success: 2000 tests passed in 2005 cycles taking 58.453178 seconds
[info] [58.365] RAN 2000 CYCLES PASSED


In [ ]:
val taps = Seq.fill(i)(r())

val goldenModel = new ScalaFirFilter(taps)

Driver(() => new MyManyElementFir(taps, 32)) {
    c => new PeekPokeTester(c) {
        for(i <- 0 until 100) {
            val input = r()

            val goldenModelResult = goldenModel.poke(input)

            poke(c.io.in, input)

            expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

            step(1)
        }
    }
}

In [ ]:
class MyManyDynamicElementFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })

  // Reference solution
  val regs = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until length) {
      if(i == 0) regs += io.in
      else       regs += RegNext(regs(i - 1), 0.U)
  }
  
  val muls = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until length) {
      muls += regs(i) * io.consts(i)
  }

  val scan = mutable.ArrayBuffer[UInt]()
  for(i <- 0 until length) {
      if(i == 0) scan += muls(i)
      else scan += muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

## Add run-time configurable taps to our FIR
The following code adds and additional `consts` vector to the IO of our FIR generator which allows the coefficients to be changed externally after circuit generation.
This is done with the Chisel collection type `Vec`.
`Vec` supports many of the scala collection methods but it can only contain Chisel hardware elements.
`Vec` should only be used in situations where ordinary Scala collections won't work.  
Basically this is in one of two situations.
1. You need a collection of elements in a Bundle, typically a Bundle that will be used as IO.
1. You need to access the collection via an index that is part of the hardware.


In [ ]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })


  // Reference solution
  val regs = RegInit(Vec.fill(length - 1)(0.U(8.W)))
  for(i <- 0 until length - 1) {
      if(i == 0) regs(i) := io.in
      else       regs(i) := regs(i - 1)
  }
  
  val muls = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) muls(i) := io.in * io.consts(i)
      else       muls(i) := regs(i - 1) * io.consts(i)
  }

  val scan = Wire(Vec(length, UInt(8.W)))
  for(i <- 0 until length) {
      if(i == 0) scan(i) := muls(i)
      else scan(i) := muls(i) + scan(i - 1)
  }

  io.out := scan(length - 1)
}

In [ ]:
class MyManyDynamicElementVecFir(length: Int) extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(8.W))
    val out = Output(UInt(8.W))
    val consts = Input(Vec(length, UInt(8.W)))
  })
  
  val taps = Seq(io.in) ++ Seq.fill(io.consts.length - 1)(RegInit(0.U(8.W)))
  taps.zip(taps.tail).foreach { case (a, b) => b := a }

  io.out := taps.zip(io.consts).map { case (a, b) => a * b }.reduce(_ + _)
}

In [ ]:
val goldenModel = new ScalaFirFilter(Seq(1, 1, 1, 1))

Driver(() => new MyManyDynamicElementVecFir(4)) {
    c => new PeekPokeTester(c) {
        poke(c.io.consts(0), 1)
        poke(c.io.consts(1), 1)
        poke(c.io.consts(2), 1)
        poke(c.io.consts(3), 1)
        for(i <- 0 until 100) {
            val input = scala.util.Random.nextInt(8)
          
            val goldenModelResult = goldenModel.poke(input)

            poke(c.io.in, input)
          
            expect(c.io.out, goldenModelResult, s"i $i, input $input, gm $goldenModelResult, ${peek(c.io.out)}")

            step(1)
        }
    }
}